## anthropic_few_shot_prompt

In [9]:
import os
import requests
import json
from dotenv import load_dotenv

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()
api_key = os.getenv("ANTHROPIC_API_KEY")

In [10]:

def anthropic_few_shot_prompt(api_key, task_description, examples, input_data, model="claude-3-5-sonnet-20240620", max_tokens=1024, temperature=0.7):
    """
    Generates a few-shot prompt using the provided examples and task context with the Anthropic API.

    Parameters:
    api_key (str): The API key for accessing the Anthropic API.
    task_description (str): Description of the task to be performed.
    examples (list): List of few-shot examples, each example being a dictionary with 'input' and 'output'.
    input_data (str): Input data for which we want to generate a response.
    model (str): The model to use for text generation (default is "claude-3-5-sonnet-20240620").
    max_tokens (int): The maximum number of tokens in the generated response (default is 1024).
    temperature (float): Sampling temperature to control the creativity of the model (default is 0.7).

    Returns:
    str: Response generated by the Anthropic API.
    """
    url = "https://api.anthropic.com/v1/messages"
    
    headers = {
        "x-api-key": api_key,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    # Format the examples into a string
    formatted_examples = "\n\n".join([
        f"Example {i+1}:\nInput: {example['input']}\nOutput: {example['output']}"
        for i, example in enumerate(examples)
    ])

    # Build the prompt content with task description, examples, and input data
    prompt_content = f"""
    {task_description}

    Here are some examples to guide you:

    {formatted_examples}

    Now it's your turn!

    Input: {input_data}
    Output:
    """

    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {"role": "user", "content": prompt_content}
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["content"][0]["text"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"


In [11]:
task_description = "Translate sentences from English to French."
examples = [
    {"input": "Hello, how are you?", "output": "Bonjour, comment ça va ?"},
    {"input": "Good morning!", "output": "Bonjour !"},
    {"input": "I am learning how to code.", "output": "J'apprends à coder."}
]
input_data = "What is your name?"


In [12]:
generated_output = anthropic_few_shot_prompt(api_key, task_description, examples, input_data, temperature=0.7, max_tokens=100)
print("Generated Output:", generated_output)


Generated Output: Quel est votre nom ?
